In [ ]:
# 練習：靜態擷取貓貓桌布庫
import requests, re, pprint, os
from bs4 import BeautifulSoup
mainurl = 'http://big5.wallcoo.com/'#貓貓桌布庫首頁
suburl = 'nature/Magic_Landscapes_by_Michael_Breitung/index.html'#想要抓取的子網頁
response = requests.get(mainurl+suburl)
response.encoding = response.apparent_encoding
folderPath = 'Wallpaper_folder'# 建立儲存圖片的資料夾，可自訂名稱
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
List=[]
soup = BeautifulSoup(response.text, "lxml") 
for a in soup.select('div#right li a.thumbnailtitle'):
    link=(a['href'])#連結名稱
    List.append({
        "Link": link.replace("html","jpg")
    })
for index in List:
    Data = str(mainurl+suburl.replace("index.html","")+"wallpapers/1920x1200/"+index["Link"]) #生成完整網址
    print(Data)
    img = requests.get(Data)  # 下載圖片
    with open("{}/".format(folderPath)+index["Link"], "wb") as file:  # 開啟資料夾及命名圖片檔
        file.write(img.content)  # 寫入圖片的二進位碼

In [ ]:
# 練習：動態擷取貓貓桌布庫
import requests, re, os
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

folderPath2 = 'Wallpaper_folder2'# 建立儲存圖片的資料夾，可自訂名稱
if not os.path.exists(folderPath2):
    os.makedirs(folderPath2)
    
mainurl = 'http://big5.wallcoo.com/'#貓貓桌布庫首頁
suburl = 'nature/Daily_Snap_Shot_12_Nature_Wallpapers/index.html'#想要抓取的子網頁
response = requests.get(mainurl+suburl)
response.encoding = response.apparent_encoding

driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get(mainurl+suburl)

List=[]
urllist=[]

select = Select(driver.find_element_by_name('indexlist'))
for op in select.options:
    urllist.append({
        "url":mainurl+suburl.replace("index.html","")+op.get_attribute('value')
    })

for index, obj in enumerate(urllist):#換頁
    try:
        res = requests.get(obj["url"])
        res.encoding = res.apparent_encoding
        soup = BeautifulSoup(res.text, "lxml") 
        for a in soup.select('div#right li a.thumbnailtitle'):
            link=(a['href'])#連結名稱
            List.append({
                "Link": link.replace("html","jpg")
            })
            print(a)
    except:
        print("Woops,something wrong！\>'0'</")
    sleep(2) #等待頁面loading完成
    
    driver.find_element(By.CSS_SELECTOR, 'a.navigationtext').click() #前往下一頁
    
print("準備開始下載...有",len(List),"張圖")
for index in List:
    Data = str(mainurl+suburl.replace("index.html","")+"wallpapers/1920x1200/"+index["Link"])#生成完整網址
    img = requests.get(Data)  # 下載圖片
    with open("{}/".format(folderPath2)+index["Link"], "wb") as file:  # 開啟資料夾及命名圖片檔
        file.write(img.content)  # 寫入圖片的二進位碼
print("下載完成！")